In [ ]:
# Author: Arthur BARREAU
# Date: 2025-07-15
# R version: 4.3.3
# Description: Downloads NSIDC daily sea ice data, classifies presence, aggregates 
#              by subarea, and exports a masked raster map with labels and borders.
# Source data: https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/
# ==============================================================================
# 0. PACKAGE & LIBRARY INSTALLATION ----
# ==============================================================================

system("conda install -y r-sf=1.0_20 r-terra=1.8_42 r-fields")

library(terra)
library(dplyr)
library(stringr)
library(RColorBrewer)
library(sf)
library(jsonlite)
library(lubridate)
library(fields)

options(timeout = 3600)

# ==============================================================================
# 1. VARIABLES ----
# ==============================================================================

months_days <- list(
  Jan = 31, Feb = 28, Mar = 31, Apr = 30, May = 31, Jun = 30, 
  Jul = 31, Aug = 31, Sep = 30, Oct = 31, Nov = 30, Dec = 31
)

json_data <- fromJSON("galaxy_inputs/galaxy_inputs.json")


subarea       <- json_data$subarea
area_list     <- trimws(unlist(strsplit(subarea, ",")))
year          <- as.numeric(json_data$year)
month         <- match(json_data$month, names(months_days))
m             <- json_data$multiplier
text_title    <- json_data$title
Zone_ASD      <- json_data$ZoneXASD
Label_ASD     <- json_data$LabelXASD
legend_chose  <- json_data$legendXchose
nb_months     <- json_data$nbXmonths
bbox          <- json_data$bbox
png           <- json_data$png
output_folder <- sprintf("%s_tif_daily", year)
dir.create(output_folder, showWarnings = FALSE, recursive = TRUE)

# ==============================================================================
# 2. FUNCTION: DOWNLOAD TIFF FILES ----
# ==============================================================================

download_tiff_files <- function(path, year, month) {
  base_url <- "https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/"
  months <- c("01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
              "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec")
  start_date <- as.Date(sprintf("%04d-%02d-01", year, month))
  dates <- seq.Date(start_date, by = "month", length.out = nb_months)
    
  first_date <- format(dates[1], "%Y-%m")
  last_date <- format(dates[length(dates)], "%Y-%m")
  file_name <- if (png) {
      sprintf("outputs/collection/Fig4_%s_to_%s.png", first_date, last_date)
    } else {
      sprintf("outputs/collection/Fig4_%s_to_%s.pdf", first_date, last_date)
    }
    
  for (date in dates) {
    date_obj <- as.Date(date, origin = "1970-01-01")
    year_str <- year(date_obj)
    month_num <- month(date_obj)
    month_folder <- months[month_num]
    num_days <- days_in_month(date_obj)

    for (day in 1:num_days) {
      date_str <- sprintf("S_%04d%02d%02d_concentration_v3.0.tif", year_str, month_num, day)
      url <- paste0(base_url, year_str, "/", month_folder, "/", date_str)
      destfile <- file.path(path, date_str)
      tryCatch({
        download.file(url, destfile, mode = "wb")
        if (!file.exists(destfile) || file.info(destfile)$size == 0) {
          warning(sprintf("Fichier vide ou non créé pour le %s", date_str))
        }
      }, error = function(e) {
        message(sprintf("Échec du téléchargement pour le %s : %s", date_str, e$message))
      })
    }
  }
    return(file_name)
}

# ==============================================================================
# 3. MAIN: length(tif_files))LOAD AND STACK TIFF FILES ----
# ==============================================================================

file_name <- download_tiff_files(output_folder, year, month)
tif_files <- list.files(output_folder, full.names = TRUE)

aggregate_ice_days <- function(tif_files){
  stacked_rasters <- rast(tif_files)
  stacked_rasters[is.na(stacked_rasters)] <- 0
  reclassified_raster <- ifel(stacked_rasters >= 150 & stacked_rasters <= 1000, 1, 0)
  sum_raster <- sum(reclassified_raster, na.rm = TRUE)
  return(sum_raster)
}

# ==============================================================================
# 4. LOAD ASD SHAPEFILE ----
# ==============================================================================

dir.create("asd", showWarnings = FALSE)

asd_urls <- paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.", c("shp", "shx", "dbf", "prj"))
lapply(seq_along(asd_urls), function(i) download.file(asd_urls[i], file.path("asd", basename(asd_urls[i])), mode = "wb"))

ASDs <- st_read("asd/asd-shapefile-EPSG6932.shp", quiet = TRUE)
default_areas <- c('883', '882', '481', '484', '482', '483', '5843a', '5843b','5852', '485', '486', '5841', '5842', '881', '5844a', '587','586', '5851', '5844b')
subareas_selected <- ASDs %>% filter(GAR_Short_ %in% default_areas)

# ==============================================================================
# 5. MASK & CROP RASTER TO SUBAREA ----
# ==============================================================================
letters_only <- str_extract(area_list, "[A-Za-z]+")
numbers_only <- str_extract(area_list, "-?\\d+")

if (is.na(letters_only[1])) {
    subareas_selected <- ASDs %>% filter(GAR_Short_ %in% area_list)
    
    if (bbox) {
        subareas_bbox <- vect(st_as_sfc(st_bbox(subareas_selected)))
        total_sum_raster <- aggregate_ice_days(tif_files)
        total_sum_raster_masked <- mask(crop(total_sum_raster, subareas_bbox), subareas_bbox)
    } else {
        # Fonction pour retirer les trous des polygones
        st_remove_holes <- function(x) {
          if (inherits(x, "sfc")) {
            x <- st_collection_extract(x, "POLYGON")
          }
          x_filled <- lapply(x, function(poly) {
            if (length(st_geometry(poly)[[1]]) > 1) {
              st_polygon(list(st_geometry(poly)[[1]][[1]]))  # garde que l'extérieur
            } else {
              poly
            }
          })
          st_sfc(x_filled, crs = st_crs(x))
        }
        
        subareas_union <- st_union(subareas_selected)
        area_filled <- st_remove_holes(subareas_union)
        total_sum_raster <- aggregate_ice_days(tif_files)
        total_sum_raster_masked <- mask(crop(total_sum_raster, vect(area_filled)), vect(area_filled))
    }
}  else{
  # --- NSWE ZONE ---
  bounds <- list(
    N = as.numeric(numbers_only[letters_only == "N"]),
    S = as.numeric(numbers_only[letters_only == "S"]),
    W = as.numeric(numbers_only[letters_only == "W"]),
    E = as.numeric(numbers_only[letters_only == "E"])
  )
    zone_extent <- ext(bounds$W, bounds$E, bounds$S, bounds$N)
    projected_zone <- ext(project(rast(ext = zone_extent, crs = "EPSG:4326"), "EPSG:6932"))
    area_vector <- NULL  # not used here
    total_sum_raster <- aggregate_ice_days(tif_files)
    total_sum_raster_masked <- mask(crop(total_sum_raster, projected_zone), projected_zone)
}
# ==============================================================================
# 6. PLOT OUTPUT MAP ----
# ==============================================================================

colors <- colorRampPalette(c("#2F133B", "#367AFC", "#00C3EA", "#03F791", "#72FE00",
                             "#FDC207", "#FF7504", "#F32106", "#7B0503"))(length(tif_files))

if (png) {
  png(file_name,width = dim(total_sum_raster_masked)[2] * m,height = dim(total_sum_raster_masked)[1] * m)   
} else {
  pdf(file_name,width = dim(total_sum_raster_masked)[2] * m,height = dim(total_sum_raster_masked)[1] * m)
    m <- m*100
}   

plot(total_sum_raster_masked, col = colors, axes = FALSE, box = FALSE, legend = FALSE)

if (Zone_ASD) {
    print("Zone ASD")
  plot(st_geometry(subareas_selected), add = TRUE, lwd = 0.5 * m, border = 'white')
}

if (Label_ASD) {
  zones <- st_read("asd/asd-shapefile-EPSG6932.shp") 
  zones_to_plot <- if (is.na(letters_only[1])) {area_list} else {default_areas}
  zones <- zones %>% filter(GAR_Short_ %in% zones_to_plot) 
  centroids <- st_centroid(st_geometry(zones))   
  text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "white", cex = 0.5 * m)
}

if (text_title != "No Title") {
  label_y <- ifelse(text_title == "Title in the Middle", 150000, 4000000)
  text(x = 350000, y = label_y, labels = sprintf("%s_%s", year, month), cex = 1 * m, font = 1, col = "white")
}

dev.off()

In [ ]:
generate_legend <- function(zlim, breaks, colors, horizontal = TRUE, png_filename = "legend.png") {
  # Define the limits of the legend
  labels_legend <- format(zlim, nsmall = 2)  # Use zlim here for labels
  n <- 5  # size factor

  # Set image dimensions based on orientation
  if (horizontal) {
    png(filename = png_filename, width = 400 * n, height = 75 * n, bg = "transparent")
    par(mar = c(2, 5, 2, 2))  # margins for horizontal legend
  } else {
    png(filename = png_filename, width = 75 * n, height = 400 * n, bg = "transparent")
    par(mar = c(5, 2, 2, 5))  # margins for vertical legend
  }

  # Plot the legend only
  fields::image.plot(
    zlim = zlim,
    breaks = breaks,
    col = colors,
    legend.only = TRUE,
    horizontal = horizontal,
    legend.width = 1.4 * n,
    legend.shrink = 0.9,
    legend.mar = 8,
    axis.args = list(
      at = zlim,  # Use zlim for tick positions
      labels = labels_legend,  # Labels based on zlim
      cex.axis = 2,
      col.axis = "black"
    )
  )

  dev.off()
}

nb_colors <- length(tif_files)
zlim <- c(0, nb_colors)
breaks <- seq(0, nb_colors, length.out = nb_colors + 1)

if(legend_chose != "No legend"){
    if(legend_chose == "Horizontal"){
        horizontal = TRUE        
    }else{
        horizontal = FALSE        
    }
    generate_legend(zlim,breaks, colors, horizontal, png_filename = "outputs/Legend_Fig4.png")    
}
